In [ ]:
import pandas as pd
import dwsemear

import postgresemear as sql  # mover os arquivos para a pasta principal

# NECESSARIO REALIZAR A CARGA EM 'responsavel' PRIMEIRO

# GERANDO DADOS

In [ ]:
dados = pd.read_csv('Acionamentos_01012019_a_30062019.csv', sep=';', encoding='latin-1', dtype=str)

In [ ]:
dados = dwsemear.limpa_nome_das_colunas(dados)
dados = dwsemear.acionamentos_csv.corrige_campo_observacoes(dados)
dados = dwsemear.coluna_cpf_para_inteiro(dados, 'CPF CNPJ')
dados = dwsemear.acionamentos_csv.geracao_coluna_datahora(dados)
dados = dwsemear.acionamentos_csv.adiciona_responsavel(dados)
dados.loc[:,'Dias atraso'] = dados.loc[:,'Dias atraso'].astype(int)

In [ ]:
####################
AcionamentoTipo = 1
####################

AcionamentoID = 0
TabelaDeEventosID = 0
Acionamento_registros = []
Evento_registros = []
for i in dados.index:
    AcionamentoID += 1
    TabelaDeEventosID += 1
    
    #Tabela de eventos
    EventoTipo = AcionamentoTipo
    EventoID = AcionamentoID
    CPF = int(dados.loc[i,'CPF CNPJ'])
    DataDoEvento = dados.loc[i,'datahora']
    Evento_registros.append( (TabelaDeEventosID, 
                              EventoTipo, 
                              EventoID, 
                              CPF, 
                              DataDoEvento))
    
    #Acionamento
    ResponsavelPeloAcionamento = int(dados.loc[i,'responsavel_acionamento'])
    MotivoDoAcionamento = str(dados.loc[i,'Motivo Contato'])
    observacoes = str(dados.loc[i,'Observa es'])
    diasAtraso = int(dados.loc[i,'Dias atraso'])        # DiasEmAtraso precisa ser adicionado porque eles nao informam o numero do contrato
    Acionamento_registros.append( (AcionamentoID, 
                                   ResponsavelPeloAcionamento, 
                                   MotivoDoAcionamento, 
                                   observacoes,
                                   diasAtraso))

# CRIACAO DAS TABELAS

In [ ]:
# Deletando as tabelas
delete_tabela = '''DROP TABLE eventos'''
sql.execute_query(delete_tabela)
delete_tabela = '''DROP TABLE acionamentos'''
sql.execute_query(delete_tabela)

In [ ]:
crie_tabela = '''CREATE TABLE eventos
            (ID BIGINT PRIMARY KEY NOT NULL,
            EventoTipo INT NOT NULL,
            EventoID INT NOT NULL,
            CPF BIGINT NOT NULL,
            DataDoEvento timestamp with time zone NOT NULL);'''
sql.execute_query(crie_tabela)

# ATENCAO
- Esse campo aqui é um snowflake. O campo ResponsavelPeloAcionamento endereca a outra tabela.
- A motivacao disso e a normalizacao 3NF, ou seja, vao ter 3 colunas a mais com o mesmo significado.
- Porem, o Redshift fala que pra ele nao tem problema. E isso quebra o esquema de estrela.
- O Kimball nao gosta disso de jeito nenhum, hehe.
- A saida para isso e fazer a query na tabela responsavel e depois usar os numeros resultantes para fazer a query nas tabelas normais. 200 QI.

In [ ]:
crie_tabela = '''CREATE TABLE acionamentos
            (ID INT PRIMARY KEY NOT NULL,
            ResponsavelPeloAcionamento INT NOT NULL,
            MotivoDoAcionamento TEXT NOT NULL,
            Observacoes TEXT NOT NULL,
            DiasEmAtrasoAcionamento INT NOT NULL);'''
sql.execute_query(crie_tabela)

In [ ]:
query = """ INSERT INTO eventos 
            (ID, EventoTipo, EventoID, CPF, DataDoEvento) 
            VALUES (%s,%s,%s,%s,%s)"""
sql.adicione_multiplos_registros(query, Evento_registros) 

In [ ]:
query = """ INSERT INTO acionamentos
            (ID, ResponsavelPeloAcionamento, MotivoDoAcionamento, Observacoes,DiasEmAtrasoAcionamento) 
            VALUES (%s,%s,%s,%s,%s)"""
sql.adicione_multiplos_registros(query, Acionamento_registros)

In [ ]:
# os dados dessas querys nao vem ordenados
dados_query = sql.query_para_pandas('''SELECT * FROM acionamentos,eventos
                      WHERE acionamentos.ID = eventos.EventoID
                      AND eventos.EventoTipo = 1''')

In [ ]:
dados_query.head(3)

In [ ]:
# os dados dessas querys nao vem ordenados
dados_query = sql.query_para_pandas('''SELECT * FROM eventos, acionamentos
                      WHERE acionamentos.ID = eventos.EventoID
                      AND eventos.EventoTipo = 1''')

In [ ]:
dados_query.to_csv('acionamentos_sql.csv', sep=';')

In [ ]:
datas_naive = [data.tz_localize(None) for data in dados_query.loc[:,4]]

In [ ]:
dados_query.loc[:,4] = datas_naive

In [ ]:
dados_query.to_csv('acionamentos_sql.csv', sep=';')